<a href="https://colab.research.google.com/github/aneeshkrishna4739/CompanyDocAnalyser_Using_Rag/blob/main/RAG_experiment.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**Install libraries**

In [1]:
! pip install langchain langchain-community openai groq tiktoken pinecone-client langchain_pinecone unstructured pdfminer==20191125 pdfminer.six==20221105 pillow_heif unstructured_inference sentence-transformers

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.2/4.2 MB 30.8 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 981.5/981.5 kB 36.2 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 48.1/48.1 kB 2.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.4/50.4 kB 1.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.2/42.2 kB 1.4 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.0/42.0 kB 1.9 MB/s eta 0:00:00
INFO: pip is looking at multiple versions of pdfplumber to determine which version is compatible with other requirements. This could take a while.
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.0/42.0 kB 2.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 40.1/40.1 kB 2.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 48.5/48.5 kB 1.6 MB/s eta 0:00:00
   ━━━━━

In [2]:
from langchain.document_loaders import UnstructuredPDFLoader, OnlinePDFLoader, WebBaseLoader, YoutubeLoader, DirectoryLoader, TextLoader, PyPDFLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from sklearn.metrics.pairwise import cosine_similarity
from langchain_pinecone import PineconeVectorStore
from langchain.embeddings import OpenAIEmbeddings
from langchain_community.embeddings import HuggingFaceEmbeddings
from google.colab import userdata
from langchain.schema import Document
from sentence_transformers import SentenceTransformer
from pinecone import Pinecone
from openai import OpenAI
import numpy as np
import tiktoken
import os
from groq import Groq

pinecone_api_key = userdata.get("PINECONE_API_KEY")
os.environ['PINECONE_API_KEY'] = pinecone_api_key

# openai_api_key = userdata.get("OPENAI_API_KEY")
# os.environ['OPENAI_API_KEY'] = openai_api_key
# openai_client = OpenAI()

groq_api_key = userdata.get("GROQ_API_KEY")
os.environ['GROQ_API_KEY'] = groq_api_key

**Initialize the HuggingFace Embeddings client**

In [3]:
embeddings = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")

<ipython-input-3-83794808db26>:1: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFaceEmbeddings``.
  embeddings = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")
/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or d

modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/10.7k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


1_Pooling/config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

In [4]:
text = "Hello my name is Aneesh"

query_result = embeddings.embed_query(text)

In [36]:
query_result[:5]

[-0.11953826248645782,
 -0.02359662391245365,
 0.0004015534068457782,
 0.021298062056303024,
 -0.11325403302907944]

In [6]:
len(query_result)

384

**Initialize the Groq client**

In [7]:
# Free Llama 3.1 API via Groq

groq_client = Groq(api_key=os.getenv('GROQ_API_KEY'))

**Calculating sentence similarity with embeddings**

In [9]:
def get_huggingface_embeddings(text, model_name="sentence-transformers/all-MiniLM-L6-v2"):
    model = SentenceTransformer(model_name)
    return model.encode(text)


def cosine_similarity_between_sentences(sentence1, sentence2):
    # Get embeddings for both sentences
    embedding1 = np.array(get_huggingface_embeddings(sentence1))
    embedding2 = np.array(get_huggingface_embeddings(sentence2))

    # Reshape embeddings for cosine_similarity function
    embedding1 = embedding1.reshape(1, -1)
    embedding2 = embedding2.reshape(1, -1)

    # print("Embedding for Sentence 1:", embedding1)
    # print("\nEmbedding for Sentence 2:", embedding2)

    # Calculate cosine similarity
    similarity = cosine_similarity(embedding1, embedding2)
    return similarity[0][0]


# Example usage
sentence1 = "I am trying to learn RAG"
sentence2 = "I am planning use RAG to my project"


similarity = cosine_similarity_between_sentences(sentence1, sentence2)
print(f"\n\nCosine similarity between '{sentence1}' and '{sentence2}': {similarity:.4f}")

/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(




Cosine similarity between 'I am trying to learn RAG' and 'I am planning use RAG to my project': 0.7176


**Load in the Data**

In [10]:
! kaggle datasets download -d ayoubcherguelaine/company-documents-dataset
! unzip company-documents-dataset.zip

Dataset URL: https://www.kaggle.com/datasets/ayoubcherguelaine/company-documents-dataset
License(s): apache-2.0
 96% 9.00M/9.34M [00:00<00:00, 38.9MB/s]
100% 9.34M/9.34M [00:00<00:00, 37.6MB/s]
Archive:  company-documents-dataset.zip
  inflating: CompanyDocuments/Inventory Report/monthly-Category/monthly-Category/StockReport_2016-07_1.pdf  
  inflating: CompanyDocuments/Inventory Report/monthly-Category/monthly-Category/StockReport_2016-07_2.pdf  
  inflating: CompanyDocuments/Inventory Report/monthly-Category/monthly-Category/StockReport_2016-07_3.pdf  
  inflating: CompanyDocuments/Inventory Report/monthly-Category/monthly-Category/StockReport_2016-07_4.pdf  
  inflating: CompanyDocuments/Inventory Report/monthly-Category/monthly-Category/StockReport_2016-07_5.pdf  
  inflating: CompanyDocuments/Inventory Report/monthly-Category/monthly-Category/StockReport_2016-07_6.pdf  
  inflating: CompanyDocuments/Inventory Report/monthly-Category/monthly-Category/StockReport_2016-07_7.pdf  
  i

In [11]:
def process_directory(directory_path):
    data = []
    for root, _, files in os.walk(directory_path):
        for file in files:

            file_path = os.path.join(root, file)
            print(f"Processing file: {file_path}")
            loader = PyPDFLoader(file_path)
            data.append({"File": file_path, "Data": loader.load()})

    return data

directory_path = "/content/CompanyDocuments"
documents = process_directory(directory_path)

Processing file: /content/CompanyDocuments/invoices/invoice_10974.pdf
Processing file: /content/CompanyDocuments/invoices/invoice_10356.pdf
Processing file: /content/CompanyDocuments/invoices/invoice_10807.pdf
Processing file: /content/CompanyDocuments/invoices/invoice_10747.pdf
Processing file: /content/CompanyDocuments/invoices/invoice_10683.pdf
Processing file: /content/CompanyDocuments/invoices/invoice_10813.pdf
Processing file: /content/CompanyDocuments/invoices/invoice_10985.pdf
Processing file: /content/CompanyDocuments/invoices/invoice_11006.pdf
Processing file: /content/CompanyDocuments/invoices/invoice_10892.pdf
Processing file: /content/CompanyDocuments/invoices/invoice_10954.pdf
Processing file: /content/CompanyDocuments/invoices/invoice_10947.pdf
Processing file: /content/CompanyDocuments/invoices/invoice_10818.pdf
Processing file: /content/CompanyDocuments/invoices/invoice_10377.pdf
Processing file: /content/CompanyDocuments/invoices/invoice_10894.pdf
Processing file: /co

In [12]:
documents[0]

{'File': '/content/CompanyDocuments/invoices/invoice_10974.pdf',
 'Data': [Document(metadata={'source': '/content/CompanyDocuments/invoices/invoice_10974.pdf', 'page': 0}, page_content='Invoice\nOrder ID: 10974\nCustomer ID: SPLIR\nOrder Date: 2018-03-25\nCustomer Details:\nContact Name: Art Braunschweiger\nAddress: P.O. Box 555\nCity: Lander\nPostal Code: 82520\nCountry: USA\nPhone: (307) 555-4680\nFax: (307) 555-6525\nProduct Details:\nProduct ID Product Name Quantity Unit Price\n63 Vegie-spread 10 43.9\n  TotalPrice 439.0\nPage 1')]}

**Initialize Pinecone**

In [14]:
# Make sure to create a Pinecone index with 384 dimensions
# You don't need to create a namespace through Pinecone, we will just define the name of the namespace here and use it later

index_name = "rag-experiment"

namespace = "company-documents"

vectorstore = PineconeVectorStore(index_name=index_name, embedding=embeddings)

In [15]:
vectorstore

**Insert data into Pinecone**

In [16]:
for document in documents[:5]:
    print(document['File'], document['Data'])

/content/CompanyDocuments/invoices/invoice_10974.pdf [Document(metadata={'source': '/content/CompanyDocuments/invoices/invoice_10974.pdf', 'page': 0}, page_content='Invoice\nOrder ID: 10974\nCustomer ID: SPLIR\nOrder Date: 2018-03-25\nCustomer Details:\nContact Name: Art Braunschweiger\nAddress: P.O. Box 555\nCity: Lander\nPostal Code: 82520\nCountry: USA\nPhone: (307) 555-4680\nFax: (307) 555-6525\nProduct Details:\nProduct ID Product Name Quantity Unit Price\n63 Vegie-spread 10 43.9\n  TotalPrice 439.0\nPage 1')]
/content/CompanyDocuments/invoices/invoice_10356.pdf [Document(metadata={'source': '/content/CompanyDocuments/invoices/invoice_10356.pdf', 'page': 0}, page_content='Invoice\nOrder ID: 10356\nCustomer ID: WANDK\nOrder Date: 2016-11-18\nCustomer Details:\nContact Name: Rita Müller\nAddress: Adenauerallee 900\nCity: Stuttgart\nPostal Code: 70563\nCountry: Germany\nPhone: 0711-020361\nFax: 0711-035428\nProduct Details:\nProduct ID Product Name Quantity Unit Price\n31 Gorgonzola 

In [17]:
# Prepare the text for embedding
document_data = []
for document in documents:

    document_source = document['Data'][0].metadata['source']
    document_content = document['Data'][0].page_content

    file_name = document_source.split("/")[-1]
    folder_names = document_source.split("/")[2:-1]

    doc = Document(
        page_content = f"<Source>\n{document_source}\n</Source>\n\n<Content>\n{document_content}\n</Content>",
        metadata = {
            "file_name": file_name,
            "parent_folder": folder_names[-1],
            "folder_names": folder_names
        }
    )
    document_data.append(doc)

In [19]:
document_data[0]

Document(metadata={'file_name': 'invoice_10974.pdf', 'parent_folder': 'invoices', 'folder_names': ['CompanyDocuments', 'invoices']}, page_content='<Source>\n/content/CompanyDocuments/invoices/invoice_10974.pdf\n</Source>\n\n<Content>\nInvoice\nOrder ID: 10974\nCustomer ID: SPLIR\nOrder Date: 2018-03-25\nCustomer Details:\nContact Name: Art Braunschweiger\nAddress: P.O. Box 555\nCity: Lander\nPostal Code: 82520\nCountry: USA\nPhone: (307) 555-4680\nFax: (307) 555-6525\nProduct Details:\nProduct ID Product Name Quantity Unit Price\n63 Vegie-spread 10 43.9\n  TotalPrice 439.0\nPage 1\n</Content>')

In [20]:
# Insert documents into Pinecone
vectorstore_from_documents = PineconeVectorStore.from_documents(
    document_data,
    embeddings,
    index_name=index_name,
    namespace=namespace
)

**Perform RAG**

In [21]:
# Initialize Pinecone
pc = Pinecone(api_key=userdata.get("PINECONE_API_KEY"),)

# Connect to your Pinecone index
pinecone_index = pc.Index(index_name)

In [22]:
query = "What are some items that Pirkko Koskitalo is likely to buy next? What incentives can I put in place to ensure he orders more?"

In [23]:
raw_query_embedding = get_huggingface_embeddings(query)

/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


In [24]:
raw_query_embedding

array([-2.37192772e-02,  5.11897393e-02, -3.76846157e-02, -3.59025188e-02,
       -1.70774478e-02,  4.99673262e-02,  5.41750118e-02,  1.63438041e-02,
       -7.25995526e-02,  1.91715304e-02,  4.41450216e-02, -1.21413413e-02,
       -5.31499051e-02,  1.80768650e-02,  4.38538678e-02, -2.18482707e-02,
        8.62548500e-02,  2.99322605e-02, -5.04300892e-02, -3.99649851e-02,
        9.35384538e-03, -9.97161716e-02,  8.71505290e-02,  1.87936903e-03,
       -2.41057333e-02, -1.08202472e-02, -1.57260206e-02, -2.97939051e-02,
        1.20479465e-02, -7.24623278e-02,  3.19638886e-02,  6.17180690e-02,
        7.11756572e-03, -5.66110983e-02,  5.71597740e-03,  8.44609085e-03,
        2.97988509e-03, -5.20805381e-02,  2.58033983e-02,  2.08102539e-02,
        3.32184136e-02, -5.05116731e-02, -5.20432591e-02,  2.99860332e-02,
        2.27520196e-03,  2.01680623e-02,  5.08307479e-03,  3.94049240e-03,
        4.68822531e-02,  1.60782076e-02, -1.24224432e-01, -3.86481136e-02,
        3.41078863e-02, -

In [25]:
top_matches = pinecone_index.query(vector=raw_query_embedding.tolist(), top_k=10, include_metadata=True, namespace=namespace)

In [26]:
top_matches

{'matches': [{'id': '1655c9a4-c50f-4a8a-b973-f400f9aa5149',
              'metadata': {'file_name': 'purchase_orders_10333.pdf',
                           'folder_names': ['CompanyDocuments',
                                            'PurchaseOrders'],
                           'parent_folder': 'PurchaseOrders',
                           'text': '<Source>\n'
                                   '/content/CompanyDocuments/PurchaseOrders/purchase_orders_10333.pdf\n'
                                   '</Source>\n'
                                   '\n'
                                   '<Content>\n'
                                   'Purchase Orders\n'
                                   'Order ID Order Date Customer Name\n'
                                   '10333 2016-10-18 Pirkko Koskitalo\n'
                                   'Products\n'
                                   'Product ID: Product: Quantity: Unit '
                                   'Price:\n'
                     

In [27]:
contexts = [item['metadata']['text'] for item in top_matches['matches']]

In [28]:
contexts

["<Source>\n/content/CompanyDocuments/PurchaseOrders/purchase_orders_10333.pdf\n</Source>\n\n<Content>\nPurchase Orders\nOrder ID Order Date Customer Name\n10333 2016-10-18 Pirkko Koskitalo\nProducts\nProduct ID: Product: Quantity: Unit Price:\n14 Tofu 10 18.6\n21 Sir Rodney's Scones 10 8\n71 Flotemysost 40 17.2\nPage 1\n</Content>",
 '<Source>\n/content/CompanyDocuments/PurchaseOrders/purchase_orders_10583.pdf\n</Source>\n\n<Content>\nPurchase Orders\nOrder ID Order Date Customer Name\n10583 2017-06-30 Pirkko Koskitalo\nProducts\nProduct ID: Product: Quantity: Unit Price:\n29 Thüringer Rostbratwurst 10 123.79\n60 Camembert Pierrot 24 34\n69 Gudbrandsdalsost 10 36\nPage 1\n</Content>',
 '<Source>\n/content/CompanyDocuments/PurchaseOrders/purchase_orders_10412.pdf\n</Source>\n\n<Content>\nPurchase Orders\nOrder ID Order Date Customer Name\n10412 2017-01-13 Pirkko Koskitalo\nProducts\nProduct ID: Product: Quantity: Unit Price:\n14 Tofu 20 18.6\nPage 1\n</Content>',
 "<Source>\n/content/C

In [29]:
augmented_query = "<CONTEXT>\n" + "\n\n-------\n\n".join(contexts[ : 10]) + "\n-------\n</CONTEXT>\n\n\n\nMY QUESTION:\n" + query

In [31]:
print(augmented_query)

<CONTEXT>
<Source>
/content/CompanyDocuments/PurchaseOrders/purchase_orders_10333.pdf
</Source>

<Content>
Purchase Orders
Order ID Order Date Customer Name
10333 2016-10-18 Pirkko Koskitalo
Products
Product ID: Product: Quantity: Unit Price:
14 Tofu 10 18.6
21 Sir Rodney's Scones 10 8
71 Flotemysost 40 17.2
Page 1
</Content>

-------

<Source>
/content/CompanyDocuments/PurchaseOrders/purchase_orders_10583.pdf
</Source>

<Content>
Purchase Orders
Order ID Order Date Customer Name
10583 2017-06-30 Pirkko Koskitalo
Products
Product ID: Product: Quantity: Unit Price:
29 Thüringer Rostbratwurst 10 123.79
60 Camembert Pierrot 24 34
69 Gudbrandsdalsost 10 36
Page 1
</Content>

-------

<Source>
/content/CompanyDocuments/PurchaseOrders/purchase_orders_10412.pdf
</Source>

<Content>
Purchase Orders
Order ID Order Date Customer Name
10412 2017-01-13 Pirkko Koskitalo
Products
Product ID: Product: Quantity: Unit Price:
14 Tofu 20 18.6
Page 1
</Content>

-------

<Source>
/content/CompanyDocuments

In [32]:
system_prompt = f"""You are an expert at understanding and analyzing company data - particularly shipping orders, purchase orders, invoices, and inventory reports.

Answer any questions I have, based on the data provided. Always consider all of the context provided when forming a response.
"""

llm_response = groq_client.chat.completions.create(
    model="llama-3.1-70b-versatile",
    messages=[
        {"role": "system", "content": system_prompt},
        {"role": "user", "content": augmented_query}
    ]
)

response = llm_response.choices[0].message.content

In [33]:
print(response)

Based on the provided purchase orders data, we can identify patterns and preferences of Pirkko Koskitalo. Some items that Pirkko Koskitalo is likely to buy next include:

1. Flotemysost: Pirkko Koskitalo has ordered this product three times (Order ID: 10320, 10333, 10455), with quantities of 30, 40, and 30 units, respectively.
2. Perth Pasties: These were ordered in three different order IDs (10416, 10437, and 10455) with varying quantities (10, 15, and 50 units, respectively), indicating a potential long-term interest in this product.
3. Tofu (or variations of it): Pirkko Koskitalo has ordered Tofu twice (Order ID: 10333, 10412) and Longlife Tofu once (Order ID: 10781), totaling 65 units over three orders.
4. Gnocchi di nonna Alice: Ordered twice in order IDs 10526 and 10781 with quantities of 30 and 20 units, respectively.

Incentives to ensure Pirkko Koskitalo orders more can include:

1. **Discounted prices**: Offer discounts on large orders or for repeat purchases of specific item

**Putting it all together**

In [34]:
def perform_rag(query):
    raw_query_embedding = get_huggingface_embeddings(query)

    query_embedding = np.array(raw_query_embedding)

    top_matches = pinecone_index.query(vector=query_embedding.tolist(), top_k=10, include_metadata=True, namespace=namespace)

    # Get the list of retrieved texts
    contexts = [item['metadata']['text'] for item in top_matches['matches']]

    augmented_query = "<CONTEXT>\n" + "\n\n-------\n\n".join(contexts[ : 10]) + "\n-------\n</CONTEXT>\n\n\n\nMY QUESTION:\n" + query

    # Modify the prompt below as need to improve the response quality
    system_prompt = f"""You are an expert at understanding and analyzing company data - particularly shipping orders, purchase orders, invoices, and inventory reports.

    Answer any questions I have, based on the data provided. Always consider all parts of the context provided when forming a response.
    """

    res = groq_client.chat.completions.create(
        model="llama-3.1-70b-versatile", # llama-3.1-70b-versatile
        messages=[
            {"role": "system", "content": system_prompt},
            {"role": "user", "content": augmented_query}
        ]
    )

    return res.choices[0].message.content

In [35]:
response = perform_rag("What are some trends with Ricardo Adocicados purchase orders?")

print(response)

/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


Analyzing the provided shipping orders, we can identify some trends related to Ricardo Adocicados' purchase orders. 

1. **Frequency of Purchases**: Ricardo Adocicados seems to be placing frequent orders. There are multiple orders made throughout 2016, 2017, and 2018, indicating a steady demand for the products.

2. **Product Variety**: Ricardo Adocicados orders a variety of products, which can be seen in the provided shipping orders. The customer has ordered at least 21 distinct products across different categories, including beverages, baked goods, meats, etc.

3. **Favorite Products**: Some products are repeatedly ordered, such as:
   - Chang (appear in 3 orders: 10622, 10563 is without quantity for this customer but was supposed to get other items and 10813)
   - Teatime Chocolate Biscuits (appear in 2 orders: 10447, 10299)
   - Spegesild (appear in 2 orders: 10813, 10287)
   - Pavlova (appear in 2 orders: 10877, 10287)
   This trend indicates the customer might have a relatively c